In [1]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("mini-contest/dataset-agent.csv")

# Display the first few rows of the DataFrame
print(df.head(1))

   id                                          project_a  \
0   2  https://github.com/prettier-solidity/prettier-...   

                                    project_b  weight_a  weight_b  \
0  https://github.com/nomicfoundation/hardhat  0.101669  0.898331   

   agent_weight_a  agent_weight_b  
0             NaN             NaN  


In [2]:
# force a fresh import of the module, otherwise you need to restart the ipython kernel whenever you make changes to the deepfunding.py file
import importlib
import deepfunding
importlib.reload(deepfunding)

from deepfunding import run_comparison
from tools.data_collection import fetch_repo_metrics

# Initialize agent weight columns if they don't exist
if 'agent_weight_a' not in df.columns:
    df['agent_weight_a'] = None
if 'agent_weight_b' not in df.columns:
    df['agent_weight_b'] = None

for i in range(len(df)):
    if pd.notnull(df.at[i, 'agent_weight_a']) and pd.notnull(df.at[i, 'agent_weight_b']):
        continue

    repo_a_url = df.iloc[i]["project_a"]
    repo_b_url = df.iloc[i]["project_b"]
    print(f"{repo_a_url} VS {repo_b_url}")
    repo_a = { "url": repo_a_url, **fetch_repo_metrics(repo_a_url) }
    repo_b = { "url": repo_b_url, **fetch_repo_metrics(repo_b_url) }

    agent_result = run_comparison(repo_a, repo_b)

    df.at[i, 'agent_weight_a'] = agent_result['weights'][repo_a_url]
    df.at[i, 'agent_weight_b'] = agent_result['weights'][repo_b_url]
    
    print(f"Agent: {df.iloc[i]['agent_weight_a']} {df.iloc[i]['agent_weight_b']}")
    print(f"Human: {df.iloc[i]['weight_a']} {df.iloc[i]['weight_b']}")
    df.to_csv("mini-contest/dataset-agent.csv", index=False)

https://github.com/prettier-solidity/prettier-plugin-solidity VS https://github.com/nomicfoundation/hardhat


/var/folders/8s/05rz1g2j1s99dpblw0xk3m180000gp/T/ipykernel_46182/3364526265.py:22: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  repo_a = { "url": repo_a_url, **fetch_repo_metrics(repo_a_url) }


Project Analyzer Result: weight_a=0.35 weight_b=0.65 confidence=0.85 reasoning='Repo A shows lower activity metrics with only 1 active developer and fewer commits, indicating limited immediate viability. However, it has a significant funding history, hinting at past success and potential for sustainability if revitalized. Repo B, on the other hand, demonstrates robust activity, with 10 active developers and a high volume of commits, which suggests a thriving environment. The large community and recent contributions indicate stronger future potential. Given the importance of immediate viability and long-term sustainability, I assign a higher weight to Repo B.' metrics_used=['activeDeveloperCount6Months', 'closedIssueCount6Months', 'commitCount6Months', 'contributorCount', 'forkCount', 'mergedPullRequestCount6Months', 'starCount', 'total_funding_received_usd']
Funding Strategist Result: weight_a=0.3 weight_b=0.7 confidence=0.9 reasoning='Repo A (prettier-plugin-solidity) has a strong con

KeyboardInterrupt: 